In [1]:
from typing import Optional

In [2]:
from sklearn.linear_model import LogisticRegression

In [3]:
import pdpipe as pdp
from pdpipe.skintegrate import PdPipelineAndSklearnEstimator

In [4]:
# pdp.Bin?

In [5]:
class MyPipelineAndModel(PdPipelineAndSklearnEstimator):
    
    def __init__(
        self,
        savings_max_val: Optional[int] = 100,
        drop_gender: Optional[bool] = False,
        scale_numeric: Optional[bool] = False,
        ohencode_country: Optional[bool] = True,
        savings_bin_val: Optional[int] = None,
        fit_intercept: Optional[bool] = True,
    ):
        self.savings_max_val = savings_max_val
        self.drop_gender = drop_gender
        self.scale_numeric = scale_numeric
        self.ohencode_country = ohencode_country
        self.savings_bin_val = savings_bin_val
        self.fit_intercept = fit_intercept
        cols_to_drop = []
        stages = [
            pdp.ColDrop(['Name', 'Quote'], errors='ignore'),
            pdp.RowDrop({'Savings': lambda x: x > savings_max_val}),
        ]
        if savings_bin_val:
            stages.append(pdp.Bin({'Savings': [savings_bin_val]}, drop=False))
            stages.append(pdp.Encode('Savings_bin'))
        if scale_numeric:
            stages.append(pdp.Scale('MinMaxScaler'))
        if drop_gender:
            cols_to_drop.append('Gender')
        else:
            stages.append(pdp.Encode('Gender'))
        if ohencode_country:
            stages.append(pdp.OneHotEncode('Country'))
        else:
            cols_to_drop.append('Country')
        stages.append(pdp.ColDrop(cols_to_drop, errors='ignore'))
        pline = pdp.PdPipeline(stages)
        model = LogisticRegression(fit_intercept=fit_intercept)
        super().__init__(pipeline=pline, estimator=model)

In [6]:
mp = MyPipelineAndModel(
    savings_max_val=101,
    drop_gender=True,
    scale_numeric=True,
    ohencode_country=True,
    savings_bin_val=1,
    fit_intercept=True,
)

In [7]:
mp

<PdPipeline -> LogisticRegression>

In [8]:
mp.pipeline

A pdpipe pipeline:
[ 0]  Drop columns Name, Quote
[ 1]  Drop rows in columns Savings by conditions
[ 2]  Bin Savings by [1].
[ 3]  Encode Savings_bin
[ 4]  Scale columns Columns of dtypes <class 'numpy.number'>
[ 5]  One-hot encode Country
[ 6]  Drop columns Gender

In [9]:
mp.estimator

LogisticRegression()

In [10]:
mp.score?

Signature: mp.score(X, y=None)
Docstring: <no docstring>
File:      ~/clones/pdpipe/pdpipe/skintegrate.py
Type:      method


## Test our custom estimator checks out

In [11]:
from sklearn.utils.estimator_checks import check_estimator

In [12]:
# check_estimator(mp)

## Train-test for the pipeline

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(
    data=[
        [23, 'Jo', 'M', True, 0.07, 'USA', 'Living life to its fullest'],
        [52, 'Regina', 'F', False, 0.26, 'Germany', 'I hate cats'],
        [23, 'Dana', 'F', True, 0.3, 'USA', 'the pen is mightier then the sword'],
        [25, 'Bo', 'M', False, 2.3, 'Greece', 'all for one and one for all'],
        [80, 'Richy', 'M', False, 100.2, 'Finland', 'I gots the dollarz'],
        [60, 'Paul', 'M', True, 1.87, 'Denmark', 'blah'],
        [44, 'Derek', 'M', True, 1.1, 'Denmark', 'every life is precious'],
        [72, 'Regina', 'F', True, 7.1, 'Greece', 'all of you get off my porch'],
        [50, 'Jim', 'M', False, 0.2, 'Germany', 'boy do I love dogs and cats'],
        [80, 'Wealthus', 'F', False, 123.2, 'Finland', 'me likey them moniez'],
    ],
    columns=['Age', 'Name', 'Gender', 'Smoking', 'Savings', 'Country', 'Quote'],
)

In [15]:
df

,Age,Name,Gender,Smoking,Savings,Country,Quote
0,23,Jo,M,True,0.07,USA,Living life to its fullest
1,52,Regina,F,False,0.26,Germany,I hate cats
2,23,Dana,F,True,0.30,USA,the pen is mightier then the sword
3,25,Bo,M,False,2.30,Greece,all for one and one for all
4,80,Richy,M,False,100.20,Finland,I gots the dollarz
5,60,Paul,M,True,1.87,Denmark,blah
6,44,Derek,M,True,1.10,Denmark,every life is precious
7,72,Regina,F,True,7.10,Greece,all of you get off my porch
8,50,Jim,M,False,0.20,Germany,boy do I love dogs and cats
9,80,Wealthus,F,False,123.20,Finland,me likey them moniez


In [16]:
mp.pipeline(df)

,Age,Smoking,Savings,Savings_bin,Country_Finland,Country_Germany,Country_Greece,Country_USA
0,0.000000,True,0.000000,1.0,0,0,0,1
1,0.508772,False,0.001898,1.0,0,1,0,0
2,0.000000,True,0.002297,1.0,0,0,0,1
3,0.035088,False,0.022271,0.0,0,0,1,0
4,1.000000,False,1.000000,0.0,1,0,0,0
5,0.649123,True,0.017977,0.0,0,0,0,0
6,0.368421,True,0.010287,0.0,0,0,0,0
7,0.859649,True,0.070209,0.0,0,0,1,0
8,0.473684,False,0.001298,1.0,0,1,0,0


In [17]:
mp.pipeline[0:4](df)

,Age,Gender,Smoking,Savings,Savings_bin,Country
0,23,M,True,0.07,1,USA
1,52,F,False,0.26,1,Germany
2,23,F,True,0.30,1,USA
3,25,M,False,2.30,0,Greece
4,80,M,False,100.20,0,Finland
5,60,M,True,1.87,0,Denmark
6,44,M,True,1.10,0,Denmark
7,72,F,True,7.10,0,Greece
8,50,M,False,0.20,1,Germany


In [18]:
x_lbls = ['Age', 'Gender', 'Savings', 'Country']

In [19]:
all_x = df[x_lbls]
all_y = df['Smoking']

In [20]:
train_df = df.iloc[0:6]
train_df

,Age,Name,Gender,Smoking,Savings,Country,Quote
0,23,Jo,M,True,0.07,USA,Living life to its fullest
1,52,Regina,F,False,0.26,Germany,I hate cats
2,23,Dana,F,True,0.30,USA,the pen is mightier then the sword
3,25,Bo,M,False,2.30,Greece,all for one and one for all
4,80,Richy,M,False,100.20,Finland,I gots the dollarz
5,60,Paul,M,True,1.87,Denmark,blah


In [21]:
train_x = train_df[x_lbls]
train_x

,Age,Gender,Savings,Country
0,23,M,0.07,USA
1,52,F,0.26,Germany
2,23,F,0.30,USA
3,25,M,2.30,Greece
4,80,M,100.20,Finland
5,60,M,1.87,Denmark


In [22]:
train_y = train_df['Smoking']
train_y

0     True
1    False
2     True
3    False
4    False
5     True
Name: Smoking, dtype: bool

In [23]:
test_df = df.iloc[6:]
test_df

,Age,Name,Gender,Smoking,Savings,Country,Quote
6,44,Derek,M,True,1.1,Denmark,every life is precious
7,72,Regina,F,True,7.1,Greece,all of you get off my porch
8,50,Jim,M,False,0.2,Germany,boy do I love dogs and cats
9,80,Wealthus,F,False,123.2,Finland,me likey them moniez


In [24]:
test_x = test_df[x_lbls]
test_x

,Age,Gender,Savings,Country
6,44,M,1.1,Denmark
7,72,F,7.1,Greece
8,50,M,0.2,Germany
9,80,F,123.2,Finland


In [25]:
test_y = test_df['Smoking']
test_y

6     True
7     True
8    False
9    False
Name: Smoking, dtype: bool

In [26]:
mp.pipeline.fit_transform(train_x)

,Age,Savings,Savings_bin,Country_Finland,Country_Germany,Country_Greece,Country_USA
0,0.000000,0.000000,1.0,0,0,0,1
1,0.508772,0.001898,1.0,0,1,0,0
2,0.000000,0.002297,1.0,0,0,0,1
3,0.035088,0.022271,0.0,0,0,1,0
4,1.000000,1.000000,0.0,1,0,0,0
5,0.649123,0.017977,0.0,0,0,0,0


In [27]:
mp.pipeline.transform(test_x)

,Age,Savings,Savings_bin,Country_Finland,Country_Germany,Country_Greece,Country_USA
6,0.368421,0.010287,0.0,0,0,0,0
7,0.859649,0.070209,0.0,0,0,1,0
8,0.473684,0.001298,1.0,0,1,0,0


## GridSearchCV

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
gcv = GridSearchCV(
    estimator=mp,
    param_grid={
        'savings_max_val': [99, 101],
        'scale_numeric': [True, False],
        'drop_gender': [True, False],
        'ohencode_country': [True, False],
    },
    cv=3,
)

In [30]:
gcv

GridSearchCV(cv=3,
             ('estimator', <PdPipeline -> LogisticRegression>),
             param_grid={'drop_gender': [True, False],
                         'ohencode_country': [True, False],
                         'savings_max_val': [99, 101],
                         'scale_numeric': [True, False]})

In [31]:
all_x

,Age,Gender,Savings,Country
0,23,M,0.07,USA
1,52,F,0.26,Germany
2,23,F,0.30,USA
3,25,M,2.30,Greece
4,80,M,100.20,Finland
5,60,M,1.87,Denmark
6,44,M,1.10,Denmark
7,72,F,7.10,Greece
8,50,M,0.20,Germany
9,80,F,123.20,Finland


In [32]:
all_x.shape

(10, 4)

In [33]:
all_y.shape

(10,)

In [34]:
mp.score

<bound method PdPipelineAndSklearnEstimator.score of <PdPipeline -> LogisticRegression>>

In [35]:
gcv.fit(all_x, all_y)

GridSearchCV(cv=3,
             ('estimator', <PdPipeline -> LogisticRegression>),
             param_grid={'drop_gender': [True, False],
                         'ohencode_country': [True, False],
                         'savings_max_val': [99, 101],
                         'scale_numeric': [True, False]})

In [36]:
gcv

GridSearchCV(cv=3,
             ('estimator', <PdPipeline -> LogisticRegression>),
             param_grid={'drop_gender': [True, False],
                         'ohencode_country': [True, False],
                         'savings_max_val': [99, 101],
                         'scale_numeric': [True, False]})

In [37]:
gcv.cv_results_

{'mean_fit_time': array([0.01805862, 0.02602871, 0.01143765, 0.01497038, 0.01344951,
        0.01279736, 0.01329573, 0.01088969, 0.01029619, 0.01027075,
        0.01030358, 0.01006969, 0.01032559, 0.00969656, 0.01018016,
        0.01164174]),
 'std_fit_time': array([2.18801548e-03, 1.98170580e-02, 2.25330743e-04, 1.24616889e-03,
        4.48451862e-04, 2.54838793e-04, 1.36396307e-03, 9.32612305e-04,
        1.79914724e-04, 1.08602622e-04, 9.75366752e-05, 5.21974790e-04,
        3.32276720e-04, 2.21513857e-04, 4.10991280e-04, 1.38408821e-03]),
 'mean_score_time': array([0.01330503, 0.01307933, 0.00831469, 0.01016466, 0.01000388,
        0.01001596, 0.01021091, 0.00968703, 0.00961073, 0.00952029,
        0.00930103, 0.00886997, 0.00903908, 0.00896827, 0.00967216,
        0.00930333]),
 'std_score_time': array([1.13176156e-03, 6.24168155e-03, 2.63902576e-04, 6.38240356e-04,
        4.58783486e-04, 6.55998340e-04, 1.96695312e-04, 1.49338914e-04,
        5.19397646e-04, 5.46878381e-05, 1.20

In [38]:
gcv.best_estimator_

<PdPipeline -> LogisticRegression>

In [39]:
gcv.best_score_

0.4166666666666667

In [40]:
gcv.best_params_

{'drop_gender': True,
 'ohencode_country': True,
 'savings_max_val': 99,
 'scale_numeric': True}

In [41]:
post_x = mp.pipeline.fit_transform(all_x)
post_x

,Age,Savings,Savings_bin,Country_Finland,Country_Germany,Country_Greece,Country_USA
0,0.000000,0.000000,1.0,0,0,0,1
1,0.508772,0.001898,1.0,0,1,0,0
2,0.000000,0.002297,1.0,0,0,0,1
3,0.035088,0.022271,0.0,0,0,1,0
4,1.000000,1.000000,0.0,1,0,0,0
5,0.649123,0.017977,0.0,0,0,0,0
6,0.368421,0.010287,0.0,0,0,0,0
7,0.859649,0.070209,0.0,0,0,1,0
8,0.473684,0.001298,1.0,0,1,0,0


In [42]:
post_x.values

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 1.        ],
       [0.50877193, 0.00189753, 1.        , 0.        , 1.        ,
        0.        , 0.        ],
       [0.        , 0.00229701, 1.        , 0.        , 0.        ,
        0.        , 1.        ],
       [0.03508772, 0.02227105, 0.        , 0.        , 0.        ,
        1.        , 0.        ],
       [1.        , 1.        , 0.        , 1.        , 0.        ,
        0.        , 0.        ],
       [0.64912281, 0.01797663, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.36842105, 0.01028663, 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.85964912, 0.07020873, 0.        , 0.        , 0.        ,
        1.        , 0.        ],
       [0.47368421, 0.00129831, 1.        , 0.        , 1.        ,
        0.        , 0.        ]])

In [43]:
post_x.values.shape

(9, 7)

In [44]:
all_y.values

array([ True, False,  True, False, False,  True,  True,  True, False,
       False])

In [45]:
len(all_y.values)

10

In [46]:
from sklearn.metrics import fbeta_score, make_scorer
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [47]:
from pdpipe.skintegrate import pdpipe_scorer_from_sklearn_scorer

In [48]:
my_scorer = pdpipe_scorer_from_sklearn_scorer(ftwo_scorer)

In [49]:
my_scorer

<PdPipeScorer: make_scorer(fbeta_score, beta=2)>

In [50]:
gcv = GridSearchCV(
    estimator=mp,
    param_grid={
        'savings_max_val': [99, 101],
        'scale_numeric': [True, False],
        'drop_gender': [True, False],
        'ohencode_country': [True, False],
    },
    cv=3,
    scoring=my_scorer,
)

In [51]:
gcv.fit(all_x, all_y)

/Users/shaypalachy/.pyenv/versions/3.10.2/envs/py3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/shaypalachy/.pyenv/versions/3.10.2/envs/py3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/shaypalachy/.pyenv/versions/3.10.2/envs/py3/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/Users/shaypalachy/.pyenv/versions/3.10.2/envs/py3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/shaypalachy/.pyenv/versions/3.10.2/envs/py3/lib/python3.10/site-packages/sklearn/mo

GridSearchCV(cv=3,
             ('estimator', <PdPipeline -> LogisticRegression>),
             param_grid={'drop_gender': [True, False],
                         'ohencode_country': [True, False],
                         'savings_max_val': [99, 101],
                         'scale_numeric': [True, False]},
             scoring=<PdPipeScorer: make_scorer(fbeta_score, beta=2)>)

In [52]:
gcv.best_score_

nan

In [53]:
gcv.cv_results_

{'mean_fit_time': array([0.02251347, 0.01793726, 0.03731275, 0.01394049, 0.01395726,
        0.01475906, 0.01626094, 0.01384139, 0.0125436 , 0.01369166,
        0.01410397, 0.01696801, 0.01358072, 0.01508292, 0.01294494,
        0.01249854]),
 'std_fit_time': array([0.0057435 , 0.00010961, 0.0254704 , 0.00029333, 0.00030969,
        0.00028861, 0.00227177, 0.0022562 , 0.00081761, 0.00198139,
        0.00257316, 0.00047071, 0.00060362, 0.00286052, 0.00112376,
        0.00047571]),
 'mean_score_time': array([0.01710113, 0.01538428, 0.01374308, 0.01269952, 0.01259375,
        0.01395694, 0.01281134, 0.01260829, 0.01169602, 0.01266074,
        0.01401472, 0.01498961, 0.01255965, 0.01344705, 0.01407059,
        0.01144139]),
 'std_score_time': array([0.00177624, 0.0009683 , 0.00260959, 0.00061442, 0.0009796 ,
        0.00020359, 0.00178324, 0.00143696, 0.00082673, 0.00100055,
        0.0022554 , 0.00161291, 0.00056587, 0.00104515, 0.00250301,
        0.00037476]),
 'param_drop_gender': mask